# Ordinal Neural Network

In [1]:
# Importing libraries
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from scipy.stats import pearsonr, kendalltau
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score

In [2]:
project_directory = r'C:\Users\marco\OneDrive\Desktop\Final Year Project'
os.chdir(project_directory)
base_dir = os.getcwd() 

In [7]:

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0][1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    ccc = numerator / denominator
    return ccc

def pearson_correlation_coefficient(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

def kendalls_tau_coefficient(y_true, y_pred):
    return kendalltau(y_true, y_pred)[0]

def evaluate_individual_performance(X_test, Y_test, group_labels_test, ord_y_pred_score, game_name):
    evaluation_results = []
    for participant_id in np.unique(group_labels_test):
        idx = group_labels_test == participant_id
        participant_labels = Y_test[idx]
        participant_predictions = np.argmax(ord_y_pred_score[idx], axis=1)
        pcc_value = pearsonr(participant_labels, participant_predictions)[0]
        ccc_value = concordance_correlation_coefficient(participant_labels, participant_predictions)
        kendall_tau_value = kendalltau(participant_labels, participant_predictions)[0]

        evaluation_results.append({
            'Game Name': game_name,
            'Participant ID': participant_id,
            'PCC': pcc_value,
            'CCC': ccc_value,
            'KendallTau': kendall_tau_value
        })
    return pd.DataFrame(evaluation_results)

class Ordinal(tf.keras.Model):
    def __init__(self, num_classes):
        super(Ordinal, self).__init__()
        self.num_classes = num_classes
        self.output_layer = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, inputs):
        outputs = self.output_layer(inputs)
        probs = tf.concat([
            outputs[..., :1],
            outputs[..., 1:] - outputs[..., :-1],
            1 - outputs[..., -1:]
        ], axis=-1)
        return outputs, probs

def create_ordinal_model(num_classes, input_dim):
    input_layer = tf.keras.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(64, activation='relu')(input_layer)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    outputs, probabilities = Ordinal(num_classes)(x)
    model = tf.keras.Model(inputs=input_layer, outputs=[outputs, probabilities])
    model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy', None])
    return model

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

input_file = os.path.join('AGAIN Ranking Algorithms', 'Data_Percentiles', 'ordinal_logistic_regression_percentiles.csv')
df = pd.read_csv(input_file)

group_kfold = GroupKFold(n_splits=10)
games = df['[control]game'].unique()

evaluation_results = []

for game in games:
    print("\n===============================================================")
    print(f'Game: {game}')
    game_df = df[df['[control]game'] == game]
    feature_cols = [col for col in df.columns if '[general]' in col]
    X = game_df[feature_cols]
    Y = game_df['arousal_label']
    groups = game_df['[control]player_id']

    train_accuracies_game = []
    test_accuracies_game = []

    for train_index, test_index in group_kfold.split(X, Y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        group_labels_test = groups.iloc[test_index].values

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        num_features = X_train_scaled.shape[1] 
        ord_model = create_ordinal_model(num_classes=11, input_dim=num_features)

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
        history = ord_model.fit(X_train_scaled, Y_train, validation_data=(X_test_scaled, Y_test), epochs=4, batch_size=256, callbacks=[early_stopping], verbose=1)

        ord_y_pred_score, ord_y_pred_prob = ord_model.predict(X_test_scaled)
        ord_y_pred = np.argmax(ord_y_pred_prob, axis=1)
        ord_y_pred_train = np.argmax(ord_model.predict(X_train_scaled)[1], axis=1)  

        individual_results_df = evaluate_individual_performance(X_test, Y_test, group_labels_test, ord_y_pred_score, game)
        evaluation_results.append(individual_results_df)

combined_results_df = pd.concat(evaluation_results, ignore_index=True)
combined_results_df.to_csv('AGAIN Ranking Algorithms/Evaluation/Ordinal Neural Network/ONN_evaluation_results.csv', index=False)

print("Combined individual performance evaluation results saved.")



Game: Heist!

Epoch 1/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.4107 - val_loss: 2.1656
Epoch 2/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.1850 - val_loss: 2.0479
Epoch 3/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.0425 - val_loss: 1.9676
Epoch 4/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.9503 - val_loss: 1.9226
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.3758 - val_loss: 2.2265
Epoch 2/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.1681 - val_loss: 2.0989
Epoch 3/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.0331 - val_loss: 2.0159
Epoch 4/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.9459 - val_loss: 1.9699
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.3577 - val_loss: 2.2686
Epoch 2/4
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.2044 - val_loss: